In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
clf_Ada = AdaBoostClassifier(random_state=0)

In [3]:
df = pd.read_csv('./dataset_50.csv' )

In [4]:
# save_list = ['label', 'rsideventcnt', 'hotelcountyid', 'userhtlordavgamount', 'eventcnt',
#  'userhtlordcnt', 'userordcnt', 'rsideventcntin7d', 'cityeventcnt',
#  'userhtlordcntin180d', 'rsideventcntin3d', 'eventcntin7d',
#  'countyeventminprice', 'userhtlordcntin30d', 'userordavgamount', 'countyeventcnt',
#  'usercityordcnt', 'usercityordavgamount', 'hotelcommentscore', 'cityeventcntin3d',
#  'cityeventmaxprice', 'mobilesystem', 'hotelcityid', 'hotelbusinessareaid',
#  'cityeventminpricein3d', 'eventcntin3d', 'geohasheventminprice', 'userordcntin180d', 'hotelprice',
#  'usercityordcntin180d', 'citybizeventavgprice', 'citybizeventavgpricein3d', 'countyeventminpricein3d',
#  'cityeventminprice', 'membercityid', 'userhtlbrandordavgamount', 'userhtlbrandordcntin30d',
#  'geohasheventmaxprice', 'usercitybizordavgamount', 'cityeventavgprice', 'citybizeventmaxprice',
#  'houroftime', 'countyeventmaxprice', 'countyeventcntin3d', 'userordcntin30d', 'userhtlbrandordcnt',
#  'geohasheventavgprice', 'memberage', 'citybizeventcnt', 'citybizeventcntin3d', 'hotelbrandid']

In [5]:
# data = df[save_list]

In [6]:
# data.shape

In [7]:
# data.to_csv('./dataset_50.csv', index=False )

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
dftrain,dftest = train_test_split(df,test_size=0.1)
dfvalid,dftest = train_test_split(df,test_size=0.5)
xgb_train = xgb.DMatrix(dftrain.drop("label",axis = 1),dftrain[["label"]])
xgb_valid = xgb.DMatrix(dfvalid.drop("label",axis = 1),dfvalid[["label"]])
xgb_test = xgb.DMatrix(dftest.drop("label",axis = 1),dftest[["label"]])

In [10]:
num_boost_round = 200               
early_stopping_rounds = 10
# 配置xgboost模型参数
params_dict = dict()

# booster参数
params_dict['learning_rate'] = 0.01      # 学习率，通常越小越好。
params_dict['objective'] = 'binary:logistic'

# tree参数
params_dict['max_depth'] = 2             # 树的深度，通常取值在[3,10]之间
params_dict['min_child_weight']= 2       # 最小叶子节点样本权重和，越大模型越保守。
params_dict['gamma']= 0                   # 节点分裂所需的最小损失函数下降值，越大模型越保守。
params_dict['subsample']= 0.9             # 横向采样，样本采样比例，通常取值在 [0.5，1]之间 
params_dict['colsample_bytree'] = 0.85    # 纵向采样，特征采样比例，通常取值在 [0.5，1]之间 
params_dict['tree_method'] = 'hist'       # 构建树的策略,可以是auto, exact, approx, hist
#params_dict['process_type'] = 'update'
#params_dict['updater'] = 'refresh'
#params_dict['refresh_leaf'] = True

# regulazation参数 
# Omega(f) = gamma*T + reg_alpha* sum(abs(wj)) + reg_lambda* sum(wj**2)  
params_dict['reg_alpha'] = 0.0            #L1 正则化项的权重系数，越大模型越保守，通常取值在[0,1]之间。
params_dict['reg_lambda'] = 1.0           #L2 正则化项的权重系数，越大模型越保守，通常取值在[1,100]之间。

# 其他参数
params_dict['eval_metric'] = 'logloss' #'logloss'
params_dict['scale_pos_weight'] = 1      #不平衡样本时设定为正值可以使算法更快收敛。
params_dict['seed'] = 0
result = {}
watchlist = [(xgb_train, 'train'),(xgb_valid,'valid')] 
bst = xgb.train(params = params_dict, dtrain = xgb_train, num_boost_round = num_boost_round, verbose_eval= 1,evals = watchlist
                ,early_stopping_rounds=early_stopping_rounds,evals_result = result) #,xgb_model='./bst.model')

[0]	train-logloss:0.68315	valid-logloss:0.68254
[1]	train-logloss:0.67279	valid-logloss:0.67325
[2]	train-logloss:0.66505	valid-logloss:0.66452
[3]	train-logloss:0.65655	valid-logloss:0.65634
[4]	train-logloss:0.64644	valid-logloss:0.64690
[5]	train-logloss:0.63882	valid-logloss:0.63798
[6]	train-logloss:0.62754	valid-logloss:0.62871
[7]	train-logloss:0.62088	valid-logloss:0.62076
[8]	train-logloss:0.61138	valid-logloss:0.61192
[9]	train-logloss:0.60454	valid-logloss:0.60395
[10]	train-logloss:0.59559	valid-logloss:0.59579
[11]	train-logloss:0.58865	valid-logloss:0.58795
[12]	train-logloss:0.58014	valid-logloss:0.58027
[13]	train-logloss:0.57318	valid-logloss:0.57275
[14]	train-logloss:0.56514	valid-logloss:0.56556
[15]	train-logloss:0.55800	valid-logloss:0.55803
[16]	train-logloss:0.55039	valid-logloss:0.55131
[17]	train-logloss:0.54415	valid-logloss:0.54412
[18]	train-logloss:0.53675	valid-logloss:0.53736
[19]	train-logloss:0.53089	valid-logloss:0.53046
[20]	train-logloss:0.52492	val

In [11]:
y_pred_train = bst.predict(xgb_train, ntree_limit=bst.best_iteration)
y_pred_test = bst.predict(xgb_test, ntree_limit=bst.best_iteration)

/opt/conda/envs/pyspark/lib/python3.7/site-packages/xgboost/core.py:108: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


In [12]:
from sklearn.metrics import accuracy_score,precision_score,auc,roc_curve,recall_score
print("train accuarcy: %.4f%%" % ( accuracy_score(dftrain['label'],y_pred_train>0.5)*100.0))
print("test accuarcy: %.4f%%" % ( accuracy_score(dftest['label'],y_pred_test>0.5)*100.0))
print("train precision: %.4f%%" % (precision_score(dftrain['label'],y_pred_train>0.5)*100.0))
print("test precision: %.4f%%" % (precision_score(dftest['label'],y_pred_test>0.5)*100.0))
print("train recall: %.4f%%" % (recall_score(dftrain['label'],y_pred_train>0.5)*100.0))
print("test recall: %.4f%%" % (recall_score(dftest['label'],y_pred_test>0.5)*100.0))
fpr, tpr, thresholds = roc_curve(dftrain['label'], y_pred_train, pos_label=1)
print("train AUC: %.4f%%" % (auc(fpr, tpr)*100.0))
fpr, tpr, thresholds = roc_curve(dftest['label'], y_pred_test, pos_label=1)
print("test AUC: %.4f%%" % (auc(fpr, tpr)*100.0))

train accuarcy: 99.3017%
test accuarcy: 99.3027%


/opt/conda/envs/pyspark/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train precision: 0.0000%


/opt/conda/envs/pyspark/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


test precision: 0.0000%
train recall: 0.0000%
test recall: 0.0000%
train AUC: 73.8178%
test AUC: 73.8284%


In [13]:
from sklearn.model_selection import GridSearchCV
def Tuning(cv_params, other_params,x_train_array,y_train_):
    model2 = xgb.XGBClassifier(**other_params)
    optimized_GBM = GridSearchCV(estimator=model2, 
                                 param_grid=cv_params,
                                 scoring='accuracy',
                                 cv=None,n_jobs=-1 ,verbose=1)
    optimized_GBM.fit(x_train_array, y_train_)
    evalute_result = optimized_GBM.cv_results_['mean_test_score']
    #print('每轮迭代运行结果:{0}'.format(evalute_result))
    print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
    print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
    return optimized_GBM

In [17]:
feadf = pd.read_csv('./dataset_50.csv',nrows = 100000 )

In [18]:
feadf.shape

(100000, 51)

In [19]:
other_params = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'num_class': 1,
    'learning_rate':0.01,
    'eval_metric': 'auc'
}
cv_params = {
    'max_depth': [2, 4, 6],
    'min_child_weight' :[1 , 3 , 5 ],
    'subsample': [0.7,0.8,0.9],
    'colsample_bytree':[0.75,0.85,0.9],
}
opt = Tuning(cv_params,other_params,feadf.drop("label",axis = 1),feadf[["label"]])

Fitting 5 folds for each of 81 candidates, totalling 405 fits


/opt/conda/envs/pyspark/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/envs/pyspark/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


参数的最佳取值：{'colsample_bytree': 0.75, 'max_depth': 4, 'min_child_weight': 1, 'subsample': 0.7}
最佳模型得分:0.9938


In [20]:
num_boost_round = 200               
early_stopping_rounds = 10
# 配置xgboost模型参数
params_dict = dict()

# booster参数
params_dict['learning_rate'] = 0.01      # 学习率，通常越小越好。
params_dict['objective'] = 'binary:logistic'

# tree参数
params_dict['max_depth'] = 4             # 树的深度，通常取值在[3,10]之间
params_dict['min_child_weight']= 1       # 最小叶子节点样本权重和，越大模型越保守。
params_dict['gamma']= 0                   # 节点分裂所需的最小损失函数下降值，越大模型越保守。
params_dict['subsample']= 0.7             # 横向采样，样本采样比例，通常取值在 [0.5，1]之间 
params_dict['colsample_bytree'] = 0.75    # 纵向采样，特征采样比例，通常取值在 [0.5，1]之间 
params_dict['tree_method'] = 'hist'       # 构建树的策略,可以是auto, exact, approx, hist
#params_dict['process_type'] = 'update'
#params_dict['updater'] = 'refresh'
#params_dict['refresh_leaf'] = True

# regulazation参数 
# Omega(f) = gamma*T + reg_alpha* sum(abs(wj)) + reg_lambda* sum(wj**2)  
params_dict['reg_alpha'] = 0.0            #L1 正则化项的权重系数，越大模型越保守，通常取值在[0,1]之间。
params_dict['reg_lambda'] = 1.0           #L2 正则化项的权重系数，越大模型越保守，通常取值在[1,100]之间。

# 其他参数
params_dict['eval_metric'] = 'logloss' #'logloss'
params_dict['scale_pos_weight'] = 1      #不平衡样本时设定为正值可以使算法更快收敛。
params_dict['seed'] = 0
result = {}
watchlist = [(xgb_train, 'train'),(xgb_valid,'valid')] 
bst = xgb.train(params = params_dict, dtrain = xgb_train, num_boost_round = num_boost_round, verbose_eval= 1,evals = watchlist
                ,early_stopping_rounds=early_stopping_rounds,evals_result = result) #,xgb_model='./bst.model')

[0]	train-logloss:0.68316	valid-logloss:0.68254
[1]	train-logloss:0.67278	valid-logloss:0.67325
[2]	train-logloss:0.66520	valid-logloss:0.66453
[3]	train-logloss:0.65655	valid-logloss:0.65634
[4]	train-logloss:0.64641	valid-logloss:0.64684
[5]	train-logloss:0.63885	valid-logloss:0.63800
[6]	train-logloss:0.62817	valid-logloss:0.62908
[7]	train-logloss:0.62065	valid-logloss:0.62034
[8]	train-logloss:0.61141	valid-logloss:0.61197
[9]	train-logloss:0.60455	valid-logloss:0.60396
[10]	train-logloss:0.59578	valid-logloss:0.59613
[11]	train-logloss:0.58847	valid-logloss:0.58829
[12]	train-logloss:0.58040	valid-logloss:0.58073
[13]	train-logloss:0.57310	valid-logloss:0.57311
[14]	train-logloss:0.56519	valid-logloss:0.56562
[15]	train-logloss:0.55814	valid-logloss:0.55825
[16]	train-logloss:0.55029	valid-logloss:0.55110
[17]	train-logloss:0.54416	valid-logloss:0.54416
[18]	train-logloss:0.53688	valid-logloss:0.53715
[19]	train-logloss:0.53113	valid-logloss:0.53086
[20]	train-logloss:0.52422	val

In [21]:
y_pred_train = bst.predict(xgb_train, ntree_limit=bst.best_iteration)
y_pred_test = bst.predict(xgb_test, ntree_limit=bst.best_iteration)

/opt/conda/envs/pyspark/lib/python3.7/site-packages/xgboost/core.py:108: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  UserWarning


In [22]:
from sklearn.metrics import accuracy_score,precision_score,auc,roc_curve,recall_score
print("train accuarcy: %.4f%%" % ( accuracy_score(dftrain['label'],y_pred_train>0.5)*100.0))
print("test accuarcy: %.4f%%" % ( accuracy_score(dftest['label'],y_pred_test>0.5)*100.0))
print("train precision: %.4f%%" % (precision_score(dftrain['label'],y_pred_train>0.5)*100.0))
print("test precision: %.4f%%" % (precision_score(dftest['label'],y_pred_test>0.5)*100.0))
print("train recall: %.4f%%" % (recall_score(dftrain['label'],y_pred_train>0.5)*100.0))
print("test recall: %.4f%%" % (recall_score(dftest['label'],y_pred_test>0.5)*100.0))
fpr, tpr, thresholds = roc_curve(dftrain['label'], y_pred_train, pos_label=1)
print("train AUC: %.4f%%" % (auc(fpr, tpr)*100.0))
fpr, tpr, thresholds = roc_curve(dftest['label'], y_pred_test, pos_label=1)
print("test AUC: %.4f%%" % (auc(fpr, tpr)*100.0))

train accuarcy: 99.3020%
test accuarcy: 99.3030%
train precision: 83.1325%
test precision: 89.1304%
train recall: 0.0549%
test recall: 0.0588%
train AUC: 76.2615%
test AUC: 76.2824%
